# О проекте
<a id='target100'></a>

Сервис собирает публикации на всевозможные темы (блоги, статьи, галереи и т.д.) как из открытых источников, так и созданные на собственной платформе. Для каждого пользователя сервис формирует на основе рекомендательного алгоритма ленту публикаций.

Карточка - это обложка одной публикации. Кликая на неё, пользователь переходит по ссылке на материал. Каждую карточку определяют её тема и источник.

Возникло желание автоматизировать рутинную задачу. Каждую неделю менеджеры продукта нуждаются в ответах на следующие вопросы:
- Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?

Ответы на эти вопросы менеджеры хотят получать с помощью дашборда. С ними согласован макет:
![image.png](https://d.radikal.ru/d25/2103/f4/b750bdd8cd43.png)

В проекте требуется подготовить код, который
1. выгружает сырые данные из базы данных,
2. укладывает их в одну или несколько агрегирующих таблиц (если необходимо),
3. строит с их помощью графики

На основе этого кода дата-инженеры позднее напишут пайплайн. Второй и третий пункт можно выполнить как в Tableau Public, так и в том же скрипте, что делает выгрузку, с помощью библиотек dash и plotly. Попробуем оба способа. Чтобы отобразить дашборды (показать результат), в первом случае опубликуем его на сервере Tableau Public (и поделимся ссылкой), а во втором выведем в этой самой тетрадке, подключив модуль JupyterDash. 


### План:

1. [Получение данных и ознакомление с ними](#target1) 
2. [Дашборд в dash](#target2) 
3. [Дашборд в Tableau](#target3) 
8. [Резюме](#target99)

<a id='target1'></a>
## Получение данных и ознакомление с ними

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import re
import datetime
import plotly
import requests
from plotly import graph_objs as go
from plotly import figure_factory as ff
from dash.dependencies import Input, Output
from IPython.display import display
my_module_url = 'https://raw.githubusercontent.com/Artemii-Kravtsov/files/main/my_module.py'
url_dash = 'https://raw.githubusercontent.com/Artemii-Kravtsov/files/main/dash_visits.csv'
url_tableau = 'https://public.tableau.com/views/item_topics_fixedsize/sheet4'
url_heroku = 'https://sources-and-topics.herokuapp.com/'
exec(requests.get(my_module_url).text)
try:
    from jupyter_dash import JupyterDash
except:
    !pip install jupyter-dash
    from jupyter_dash import JupyterDash
try:
    from sqlalchemy import create_engine
except:
    !pip install sqlalchemy psycopg2
    from sqlalchemy import create_engine

load_from_db = False
db_config = {'user': 'user_name',
             'pwd': 'password', 
             'host': 'server.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-project-db'}

In [2]:
if load_from_db:
    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                db_config['pwd'],
                                                                db_config['host'],
                                                                db_config['port'],
                                                                db_config['db'])) 
    dash_visits = pd.io.sql.read_sql('SELECT * FROM dash_visits', con = engine) 
    dash_visits.to_csv('dash_visits.csv')
else:
    dash_visits = pd.read_csv(url_dash, index_col = 0)
    
dash_visits.dt = dash_visits.dt.astype('datetime64[m]')
display(dash_visits.head())

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [3]:
print('Длительность наблюдений:', dash_visits.dt.max() - dash_visits.dt.min())
print("Уникальных значений в поле 'dt':", dash_visits.dt.nunique())

Длительность наблюдений: 0 days 00:32:00
Уникальных значений в поле 'dt': 17


Временной диапазон в 32 минуты, но всего 17 уникальных значений в поле, отображающем время. Это значит, что на временном ряде есть минуты, которым соответствует ноль посещений. Но если аггрегировать таблицу с помощью группировки `groupby` по времени, то на графике будет всего 17 точек и ни одной точки со значением 0. Чтобы исправить это, нужно сджойнить аггрегированную таблицу с таким Series, в котором перечислены все 32 значения. Создадим этот объект Series.

In [4]:
minutes_array_full = []
time = dash_visits.dt.min()
while time < dash_visits.dt.max():
    time += pd.Timedelta(1, 'm')
    minutes_array_full.append(time)
minutes_array_full = (pd.Series(index = minutes_array_full, dtype = 'float64')
                 .reset_index().rename({'index': 'dt', 0: 'visits'}, axis = 1))

In [5]:
if dash_visits.record_id.is_unique:
    print("Значения в поле 'record_id' уникальны")
else:
    print("Значения в поле 'record_id' неуникальны")

Значения в поле 'record_id' уникальны


In [6]:
dash_visits.visits.describe()

count    30745.000000
mean        10.089673
std         19.727601
min          1.000000
25%          1.000000
50%          3.000000
75%         10.000000
max        371.000000
Name: visits, dtype: float64

В поле 'visits' нет нулей и нет неправдоподобно высоких значений.

In [7]:
for col in ['item_topic', 'source_topic', 'age_segment']:
    print(f'Уникальные значения в поле "{col}":')
    print(dash_visits[col].unique().tolist())
    dash_visits[col] = dash_visits[col].astype('category')
    print()

Уникальные значения в поле "item_topic":
['Деньги', 'Дети', 'Женская психология', 'Женщины', 'Здоровье', 'Знаменитости', 'Интересные факты', 'Искусство', 'История', 'Красота', 'Культура', 'Наука', 'Общество', 'Отношения', 'Подборки', 'Полезные советы', 'Психология', 'Путешествия', 'Рассказы', 'Россия', 'Семья', 'Скандалы', 'Туризм', 'Шоу', 'Юмор']

Уникальные значения в поле "source_topic":
['Авто', 'Деньги', 'Дети', 'Еда', 'Здоровье', 'Знаменитости', 'Интерьеры', 'Искусство', 'История', 'Кино', 'Музыка', 'Одежда', 'Полезные советы', 'Политика', 'Психология', 'Путешествия', 'Ремонт', 'Россия', 'Сад и дача', 'Сделай сам', 'Семейные отношения', 'Семья', 'Спорт', 'Строительство', 'Технологии', 'Финансы']

Уникальные значения в поле "age_segment":
['18-25', '26-30', '31-35', '36-40', '41-45', '45+']



Три поля с данными категориального типа. Всё в порядке. 

In [8]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  category      
 2   source_topic  30745 non-null  category      
 3   age_segment   30745 non-null  category      
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: category(3), datetime64[ns](1), int64(2)
memory usage: 1.0 MB


Пропущенных значений нет.

<a id='target2'></a>
## Дашборд в dash
[// вернуться к началу](#target100) 

In [9]:
# значения фильтров по умолчанию
topics_list = dash_visits.item_topic.unique().tolist()
age_groups_list = dash_visits.age_segment.unique().tolist()
min_input = ':'.join(str(dash_visits.dt.min()).split(' ')[1].split(':')[:2])
max_input = ':'.join(str(dash_visits.dt.max()).split(' ')[1].split(':')[:2])
min_max_input = ' - '.join([min_input, max_input])

# верстка дашборда
app = JupyterDash(__name__, external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div([
    html.Br(), html.Br(), 
    html.Div([
        html.Div([html.H4(children = 'Темы и источники показанных карточек', 
                     style = {'position': 'absolute', 'top': '1.6%', 'left': '1%'})], className = 'six columns'),
        html.Div([html.Label('Дата:'),
                  dcc.DatePickerRange(
                     start_date = dash_visits.dt.min(), end_date = dash_visits.dt.max(), 
                     display_format = 'DD MMM YYYY', id = 'dt_selector')], 
                     className = 'four columns', style = {'position': 'absolute', 'top': '4.6%', 'left': '48.3%'}),
        html.Div([html.Label('Время:'),
                  dcc.Input(
                     type = 'text', placeholder = min_max_input, value = min_max_input, 
                     id = 'time_selector', style = {'width': '250%'})], 
                     className = 'one column', style = {'position': 'absolute', 'top': '4.6%', 'left': '80%'})
             ], className = 'row'), 
    html.Br(), html.Br(),
 

    html.Div([
        html.Div([html.Br()], className = 'three columns'),
        html.Div([html.Label('Темы карточек:'),
                  dcc.Checklist(
                     id = 'include_all_button', labelStyle = {'display': 'block'},
                     options = [{'label': ' Показать все темы', 'value': 'All'}],
                     value = ['All'], style = {'padding-left': '35%'}),
                  dcc.Dropdown(
                     options = [{'label': x, 'value': x} for x in topics_list],
                     value = topics_list,
                     multi = True, optionHeight = 15,
                     id = 'topics_selector')], className = 'six columns'),
        html.Div([html.Label('Возрастные категории:', style = {'padding-top': '2%'}),
                  dcc.Dropdown(
                     options = [{'label': x, 'value': x} for x in age_groups_list],
                     value = age_groups_list,
                     multi = True, optionHeight = 15,
                     id = 'ages_selector', style = {'margin-top': '10%'})], className = 'three columns')
             ], className = 'row'),
    html.Br(), html.Br(),
    
    html.Div([
        html.Div([html.H6(children = 'просмотры карточек:', style = {'text-align': 'center'}),
                  dcc.Graph(id = 'abs', config = my_plotly_config(display_bar = False))], className = 'four columns'),
        html.Div([html.H6(children = 'cоотношение просмотров:', style = {'text-align': 'center'}),
                  dcc.Graph(id = 'rel', config = my_plotly_config(display_bar = False))], className = 'four columns'),
        html.Div([html.H6(children = 'соотношение по источникам:', style = {'text-align': 'center'}),
                  dcc.Graph(id = 'pie', config = my_plotly_config(display_bar = False))], className = 'four columns')
             ], className = 'row'),
    
    html.Br(),
    html.Div([html.H6(children = 'просмотры карточек (по темам и источникам):', style = {'text-align': 'center'}),
              dcc.Graph(id = 'heat', config = my_plotly_config(display_bar = False))])         
])

# подключение элементов управления к функции, рисующей графики
@app.callback([Output('include_all_button', 'value'), 
               Output('topics_selector', 'value'), 
               Output('abs', 'figure'),
               Output('rel', 'figure'),
               Output('pie', 'figure'),
               Output('heat', 'figure')],
              [Input('dt_selector', 'start_date'),
               Input('dt_selector', 'end_date'),
               Input('time_selector', 'value'),
               Input('topics_selector', 'value'),
               Input('ages_selector', 'value'),
               Input('include_all_button', 'value')])
def update_dash(start_date, end_date, time_range, topics, age_groups, button):
    # проверка того, корректно ли введено время, получение полноценных start_date и end_date
    if not re.fullmatch(r'([0-1]\d|2[0-3]):[0-5]\d - ([0-1]\d|2[0-3]):[0-5]\d', time_range):
        raise dash.exceptions.PreventUpdate
    mn, mx = time_range.split(' - ')
    start_date_complete = datetime.datetime.strptime(' '.join([str(start_date).split('T')[0], mn]), '%Y-%m-%d %H:%M')
    end_date_complete = datetime.datetime.strptime(' '.join([str(end_date).split('T')[0], mx]), '%Y-%m-%d %H:%M')
    
    # синхронизация кнопки "Показать все темы" с выпадающим списком тем
    ctx = dash.callback_context
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if input_id == 'topics_selector':
        button = [['All'] if len(topics) == len(topics_list) else []][0]
    elif input_id == 'include_all_button':
        topics = [topics_list if button else []][0]
    
    # срез данных на основе применённых фильтров
    df = dash_visits[ (dash_visits.dt >= start_date_complete) & (dash_visits.dt <= end_date_complete) 
                       & dash_visits.age_segment.isin(age_groups) & dash_visits.item_topic.isin(topics)]
    minutes_array = minutes_array_full[ (minutes_array_full.dt <= end_date_complete) &
                                        (minutes_array_full.dt >= start_date_complete) ]
    df_abs = df.groupby(['dt', 'item_topic'])['visits'].sum()
    df_rel = df_abs.div(df.groupby(['dt'])['visits'].sum()).reset_index()
    df_abs = df_abs.reset_index()

    # 1 и 2 графики
    stack_area_abs = []
    stack_area_rel = []
    topics_order = df.groupby('item_topic')['visits'].sum().sort_values(ascending = False).index
    for topic in topics_order:
        abs_vis = df_abs[ df_abs.item_topic == topic ]
        abs_vis = minutes_array.merge(abs_vis.iloc[:, [0, 2]], on = 'dt', 
                                      how = 'outer').set_index('dt').iloc[:, 1].fillna(0)
        rel_vis = df_rel[ df_rel.item_topic == topic ]
        rel_vis = minutes_array.merge(rel_vis.iloc[:, [0, 2]], on = 'dt', 
                                      how = 'outer').set_index('dt').iloc[:, 1].fillna(0)
        stack_area_abs.append(go.Scatter(x = abs_vis.index, y = abs_vis.values, stackgroup = 'one', 
                                mode = 'lines', text = ['<br>'.join([date.strftime('%d %B'), date.strftime('%H:%M'), 
                                                 f'<br>Тема: <b>{topic}</b>', f'Просмотров: <b>{int(vis)}</b>']) 
                                                 for date, vis in abs_vis.items()], 
                                hovertemplate = '%{text}<extra></extra>'))
        stack_area_rel.append(go.Scatter(x = rel_vis.index, y = rel_vis.values, stackgroup = 'one', 
                                mode = 'lines', text = ['<br>'.join([date.strftime('%d %B'), date.strftime('%H:%M'), 
                                                 f'<br>Тема: <b>{topic}</b>', 
                                                 'Доля просмотров: <b>{:.2%}</b>'.format(vis)]) 
                                                 for date, vis in rel_vis.items()], 
                                hovertemplate = '%{text}<extra></extra>'))
    layout_abs = dict(height = 300, showlegend = False, margin = dict(l = 0, r = 0, t = 30, b = 0), xaxis = dict(
                      tickangle = -90, tickmode = 'array', tickvals = abs_vis[abs_vis != 0].index, 
                      ticktext = [date.strftime('%H:%M') for date in abs_vis[abs_vis != 0].index], 
                      tickfont = {'size': 8}))
    layout_rel = layout_abs.copy()
    layout_rel.update({'yaxis': dict(range = (0, 1), tickformat = '%', tickmode = 'array', 
                                     tickvals = np.arange(0.2, 1.1, 0.2))})
    ab = go.Figure(data = stack_area_abs, layout = layout_abs)
    rel = go.Figure(data = stack_area_rel, layout = layout_rel)

    # круговая диаграмма
    pie_abs = df.groupby('source_topic')['visits'].sum().sort_values(ascending = False)
    pie_data = go.Pie(labels = pie_abs.index, values = pie_abs.values, textposition = 'inside', opacity = 0.9, hole = 0.4, 
                 hovertemplate = 'Источник:   <b>%{label}</b><br>Доля просмотров: <b>%{percent}</b><br>Просмотров: '
                 '<b>%{value}</b><extra></extra>', textfont = {'size': 12}, hoverlabel = {'align': 'right'})
    layout_pie = dict(height = 300, showlegend = False, margin = dict(l = 0, r = 0, t = 0, b = 0))
    pie = go.Figure(data = pie_data, layout = layout_pie)

    # тепловая таблица
    pivot = df.pivot_table(index = 'item_topic', columns = 'source_topic', aggfunc = 'sum', 
                           values = 'visits').fillna(0).applymap(int)
    if pivot.shape[0] != 0:
        topic_part = pivot.div(pivot.sum(axis = 1), axis = 0)
        heat = ff.create_annotated_heatmap(z = pivot.values.tolist(), x = pivot.columns.tolist(),
                                           y = pivot.index.tolist(), colorscale = 'Greys', showscale = False, 
                                           font_colors = ['black', 'white'], text = topic_part.values.tolist(),
                                           hovertemplate = 'Тема: <b>%{y}</b><br>Источник: <b>%{x}</b><br>Просмотров: '
                                                     '<b>%{z}</b><extra></extra><br><br>Доля от всех просмотров темы: '
                                                     '<b>%{text:.2%}</b>')
        for i in range(len(heat.layout.annotations)):
            heat.layout.annotations[i].font.size = 10
        heat.layout.height = 28 * len(pivot.index)
    else: 
        pivot = dash_visits.pivot_table(index = 'item_topic', columns = 'source_topic', values = 'record_id', 
                                aggfunc = 'count').applymap(lambda x: 0)
        heat = ff.create_annotated_heatmap(z = pivot.values.tolist(), x = pivot.columns.tolist(), y = pivot.index.tolist(), 
                          colorscale = 'Greys', showscale = False, hoverinfo = 'skip')
        heat.layout.height = 28 * len(pivot.index)

    return button, topics, ab, rel, pie, heat

if __name__ == '__main__':
    app.run_server(mode = 'inline')

In [10]:
print('Посмотреть на сайте Heroku: \n', url_heroku)

Посмотреть на сайте Heroku: 
 https://sources-and-topics.herokuapp.com/


<a id='target3'></a> 
## Дашборд в Tableau
[// вернуться к началу](#target100) 

Для отображения в тетрадке следовало выбрать дашборд с автоматической настройкой размера. У этой опции есть минус - на маленьких экранах графики становятся неразборчивыми, иногда сползает верстка. Поэтому если дашборд, помещённый ниже, плохо отображается, можно перейти по ссылке под ним на сайт Tableau Public. По ссылке будет дашборд с фиксированным размером, который должен корректно отображаться на любых экранах.

In [11]:
%%HTML
<div class='tableauPlaceholder' id='viz1616248004448' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;it&#47;item_topics_automatic&#47;sheet4&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='item_topics_automatic&#47;sheet4' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;it&#47;item_topics_automatic&#47;sheet4&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1616248004448');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [12]:
print('Посмотреть на сайте Tableau Public: \n', url_tableau)

Посмотреть на сайте Tableau Public: 
 https://public.tableau.com/views/item_topics_fixedsize/sheet4


<a id='target99'></a>
## Резюме
[// вернуться к началу](#target100) 

- Ссылка на дашборд, построенный в dash: https://sources-and-topics.herokuapp.com/
- Cсылка на дашборд, построенный в Tableau: https://public.tableau.com/views/item_topics_fixedsize/sheet4
- Одна запись в датасете - это не один просмотр (так как 'visits' - это количество просмотров) и не одна карточка (так как 'age_segment' - это переменная, описываюшая читателя, а не карточку. Судя по тому, что в конце взятого часа виден резкий всплекс активности, а для половины от всего времени наблюдения вообще нет записей, в базу данных с какой-то переодичностью заносятся уже агрегированные данные: результат выполнения какой-то функции. Это предположение подтверждается тем, что в данных мы не найдём две такие записи, у которых совпадают age, topic, source и время, когда они были сделаны.

In [13]:
dash_visits.duplicated(subset = ['item_topic', 'source_topic', 'age_segment', 'dt']).sum()

0

Поэтому в детализации до минут (на которой настояли при согласовании макета) может не быть никакого смысла. Если в базу заносятся аггрегированные данные за час, то в дашборде нужна почасовая детализация. 
- С нормализованной stacked area chart (второй график) удобно следить за постепенным и взаимным изменением значений на временном ряду (когда изменения взаимосвязаны). При этом желательно, чтобы выполнялись несколько условий: во-первых, должно быть заранее известно, за какими значениями следить (потому что график "шумный" - любое колебание в одном значении двигает вверх/вниз весь график, и если не знать, куда смотреть, то ничего не будет понятно), а во-вторых, когда имеется достаточный промежуток времени, чтобы проследить за трендом (менеджерам виднее, но я бы начинал от одной недели). Для текущих целей больше подошёл бы гантельный график (dumbbell chart) с нормализованными значениями - долями каждой темы от общего числа просмотров - за выбранный промежуток времени.
- Для сравнения тем по охвату аудитории отлично подходит круговая диаграмма. А тепловая таблица отлично иллюстрирует, какие сочетания тем и источников наиболее популярны, какие категории подобраны удачно и встречаются во многих сочетаниях, а какие - неудачно. 
- Tableau быстрее реагирует на фильтры, а dash даёт больше возможностей в вёрстке и в графиках. В этом проекте много фильтров и несложные, стандартные вычисления, поэтому лучше выбрать Tableau.